In [1]:
import warnings
import numpy as np
import pandas as pd
from datetime import date, datetime
from IPython.core.display import display, HTML
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))

In [2]:
matches = pd.read_csv('Matches.csv')
balls = pd.read_csv('Match Details.csv')

In [3]:
ipl = balls.merge(matches, on = 'match_id', how = 'left')
ipl['year'] = ipl['date'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').year)
ipl['date'] = pd.to_datetime(ipl['date'])
ipl = ipl.sort_values(by = ['match_id', 'innings', 'overs', 'balls'])
ipl.reset_index(inplace = True, drop = True)

In [4]:
final_score = pd.DataFrame(ipl.groupby(['match_id', 'innings'])['total_runs'].sum())
final_score.reset_index(inplace = True)
final_score.rename(columns = {'total_runs' : 'final_score'}, inplace = True)
ipl = ipl.merge(final_score, on = ['match_id', 'innings'], how = 'left')

In [5]:
total_wickets = pd.DataFrame(ipl.groupby(['match_id', 'innings'])['is_wicket'].sum())
total_wickets.reset_index(inplace = True)
total_wickets.rename(columns = {'is_wicket' : 'total_wickets'}, inplace = True)
ipl = ipl.merge(total_wickets, on = ['match_id', 'innings'], how = 'left')

In [6]:
ipl['runs'] = 0
ipl['runs'][0] = ipl['total_runs'][0]
ipl['runs'][ipl.shape[0]-1] = ipl['final_score'][ipl.shape[0]-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] == ipl['innings'][i+1]:
            ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] != ipl['match_id'][i+1]:
        ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] != ipl['innings'][i+1]:
            ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]

In [7]:
ipl['wickets'] = 0
ipl['wickets'][0] = ipl['is_wicket'][0]
ipl['wickets'][ipl.shape[0]-1] = ipl['total_wickets'][ipl.shape[0]-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] == ipl['innings'][i+1]:
            ipl['wickets'][i] = ipl['is_wicket'][i] + ipl['wickets'][i-1]
for i in range(ipl.shape[0]-1):
    if ipl['match_id'][i] != ipl['match_id'][i+1]:
        ipl['wickets'][i] = ipl['total_wickets'][i]
for i in range(ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] != ipl['innings'][i+1]:
            ipl['wickets'][i] = ipl['total_wickets'][i]

In [8]:
ipl['runs_last_5_overs'] = 0
for i in ipl.index:
    if ipl['overs'][i] <= 4:
        ipl['runs_last_5_overs'][i] = ipl['runs'][i]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] == ipl['match_id'][i-30]:
            if ipl['innings'][i] == ipl['innings'][i-30]:
                ipl['runs_last_5_overs'][i] = ipl['runs'][i] - ipl['runs'][i-30]

In [9]:
ipl['wickets_last_5_overs'] = 0
for i in ipl.index:
    if ipl['overs'][i] <= 4:
        ipl['wickets_last_5_overs'][i] = ipl['wickets'][i]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] == ipl['match_id'][i-30]:
            if ipl['innings'][i] == ipl['innings'][i-30]:
                ipl['wickets_last_5_overs'][i] = ipl['wickets'][i] - ipl['wickets'][i-30]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] != ipl['match_id'][i-1]:
            ipl['wickets_last_5_overs'][i-1] = ipl['wickets'][i-1] - ipl['wickets'][i-31]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] == ipl['match_id'][i-1]:
            if ipl['innings'][i] != ipl['innings'][i-1]:
                ipl['wickets_last_5_overs'][i-1] = ipl['wickets'][i-1] - ipl['wickets'][i-31]

In [10]:
ipl['ball_no'] = 0
ipl['extras_type'] = ipl['extras_type'].astype(str)
for i in ipl.index:
    if (ipl['overs'][i] == 0) & (ipl['balls'][i] == 1):
        ipl['ball_no'][i] = 1
for i in range(1, ipl.shape[0]):
    if (ipl['overs'][i] == 0) & (ipl['balls'][i] == 1):
        pass
    else:
        if 'wides' in ipl['extras_type'][i-1]:
            ipl['ball_no'][i] = ipl['ball_no'][i-1]
        elif 'noballs' in ipl['extras_type'][i-1]:
            ipl['ball_no'][i] = ipl['ball_no'][i-1]
        else:
            ipl['ball_no'][i] = ipl['ball_no'][i-1] + 1

In [12]:
ipl['balls'] = ipl['ball_no'] % 6

In [15]:
for i in ipl.index:
    if ipl['balls'][i] == 0:
        ipl['overs'][i] += 1

In [16]:
ipl.head(120)

,match_id,innings,overs,balls,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,target_overs,target_runs,year,city,date,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,method,players,umpire1,umpire2,playoff,knockout,final,final_score,total_wickets,runs,wickets,runs_last_5_overs,wickets_last_5_overs,ball_no
0,335982,1,0,1,SC Ganguly,BB McCullum,P Kumar,0.0,1.0,1.0,0,0,NaN,NaN,NaN,legbyes,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,1,0,1,0,1
1,335982,1,0,2,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,1,0,1,0,2
2,335982,1,0,3,BB McCullum,SC Ganguly,P Kumar,0.0,1.0,1.0,0,0,NaN,NaN,NaN,wides,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,2,0,2,0,3
3,335982,1,0,3,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,2,0,2,0,3
4,335982,1,0,4,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,2,0,2,0,4
5,335982,1,0,5,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,2,0,2,0,5
6,335982,1,1,0,BB McCullum,SC Ganguly,P Kumar,0.0,1.0,1.0,0,0,NaN,NaN,NaN,legbyes,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,3,0,3,0,6
7,335982,1,1,1,BB McCullum,SC Ganguly,Z Khan,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,3,0,3,0,7
8,335982,1,1,2,BB McCullum,SC Ganguly,Z Khan,4.0,0.0,4.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challenge

In [11]:
max_balls = pd.DataFrame(ipl.groupby(['match_id', 'innings'])['ball_no'].max())
max_balls.rename(columns = {'ball_no' : 'max_balls'}, inplace = True)
max_balls.reset_index(inplace = True)

In [12]:
ipl = ipl.merge(max_balls, on = ['match_id', 'innings'], how = 'left')

In [16]:
ipl['balls_remaining'] = 0
for i in ipl.index:
    if ipl['innings'][i] == 1:
        ipl['balls_remaining'][i] = ipl['max_balls'][i] - ipl['ball_no'][i]
    else:
        ipl['balls_remaining'][i] = (ipl['target_overs'][i] * 6) - ipl['ball_no'][i]

In [19]:
ipl['runs_scored_in_remaining_balls'] = 0
ipl['runs_required'] = 0
for i in ipl.index:
    if ipl['innings'][i] == 1:
        ipl['runs_scored_in_remaining_balls'][i] = ipl['final_score'][i] - ipl['runs'][i]
    else:
        ipl['runs_required'][i] = ipl['target_runs'][i] - ipl['runs'][i]

In [22]:
ipl.head()

,match_id,innings,overs,balls,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,target_overs,target_runs,year,city,date,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,method,players,umpire1,umpire2,playoff,knockout,final,final_score,total_wickets,runs,wickets,runs_last_5_overs,wickets_last_5_overs,ball_no,max_balls,balls_remaining,runs_scored_in_remaining_balls,runs_required
0,335982,1,0,1,SC Ganguly,BB McCullum,P Kumar,0.0,1.0,1.0,0,0,NaN,NaN,NaN,legbyes,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,1,0,1,0,1,120,119,221,0
1,335982,1,0,2,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,1,0,1,0,2,120,118,221,0
2,335982,1,0,3,BB McCullum,SC Ganguly,P Kumar,0.0,1.0,1.0,0,0,NaN,NaN,NaN,wides,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,2,0,2,0,3,120,117,220,0
3,335982,1,0,4,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,2,0,2,0,3,120,117,220,0
4,335982,1,0,5,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,0.0,0,0,NaN,NaN,NaN,nan,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,2008,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,NaN,"R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...",Asad Rauf,RE Koertzen,0,0,0,222.0,3,2,0,2,0,4,120,116,220,0


In [23]:
for col in ['team1', 'team2', 'batting_team', 'bowling_team']:
    t_list = []
    for i in ipl[col]:
        if i == 'Delhi Daredevils':
            t_list.append('Delhi Capitals')
        elif i == 'Rising Pune Supergiants':
            t_list.append('Rising Pune Supergiant')
        elif i == 'Kings XI Punjab':
            t_list.append('Punjab Kings')
        else:
            t_list.append(i)
    ipl[col] = t_list

In [24]:
t_list = []
for i in ipl['venue']:
    if i == 'M.Chinnaswamy Stadium':
        t_list.append('M Chinnaswamy Stadium')
    elif i == 'MA Chidambaram Stadium, Chepauk, Chennai':
        t_list.append('MA Chidambaram Stadium')
    elif i == 'MA Chidambaram Stadium, Chepauk':
        t_list.append('MA Chidambaram Stadium')
    elif i == 'Subrata Roy Sahara Stadium':
        t_list.append('Maharashtra Cricket Association Stadium')
    elif i == 'Sardar Patel Stadium, Motera':
        t_list.append('Narendra Modi Stadium')
    elif i == 'Narendra Modi Stadium, Ahmedabad':
        t_list.append('Narendra Modi Stadium')
    elif i == 'Wankhede Stadium, Mumbai':
        t_list.append('Wankhede Stadium')
    elif i == 'Feroz Shah Kotla':
        t_list.append('Arun Jaitley Stadium')
    elif i == 'Arun Jaitley Stadium, Delhi':
        t_list.append('Arun Jaitley Stadium')
    elif i == 'Brabourne Stadium, Mumbai':
        t_list.append('Brabourne Stadium')
    elif i == 'Punjab Cricket Association IS Bindra Stadium, Mohali':
        t_list.append('Punjab Cricket Association IS Bindra Stadium')
    elif i == 'Punjab Cricket Association Stadium, Mohali':
        t_list.append('Punjab Cricket Association IS Bindra Stadium')
    elif i == 'Rajiv Gandhi International Stadium, Uppal':
        t_list.append('Rajiv Gandhi International Stadium')
    elif i == 'Zayed Cricket Stadium, Abu Dhabi':
        t_list.append('Sheikh Zayed Stadium')
    elif i == 'Vidarbha Cricket Association Stadium, Jamtha':
        t_list.append('Vidarbha Cricket Association Stadium')
    else:
        t_list.append(i)
ipl['venue'] = t_list